In [16]:
import pandas as pd
import plotly.express as px  # (version 4.7.0 or higher)
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from dash import Dash, dash_table, dcc, html, Input, Output  # pip install dash (version 2.0.0 or higher)
import dash_bootstrap_components as dbc
from jupyter_dash import JupyterDash
import plotly.io as pio
from plotly import figure_factory as ff

In [2]:
pip install jupyter_dash

  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11429 sha256=56f9ace741b92c03684aa434a5d671f90ede5fddc4b9f95480aa93659dba4f70
  Stored in directory: c:\users\u6033919\appdata\local\pip\cache\wheels\c4\a7\48\0a434133f6d56e878ca511c0e6c38326907c0792f67b476e56
Successfully built retrying
Note: you may need to restart the kernel to use updated packages.


In [3]:
# -- Import and clean data (importing csv into pandas)
# df = pd.read_csv("intro_bees.csv")
#df = pd.read_csv("../Documents/preprocessed.csv")
ig_deals_cleaned = pd.read_csv("../Documents/preprocessed.csv")
quarterly_stats = ig_deals_cleaned.groupby('quarter').agg({'Size_m': 'sum'}).reset_index().set_index('quarter')


sectors_q = ig_deals_cleaned[(ig_deals_cleaned['quarter'] == '2021Q1') | (ig_deals_cleaned['quarter'] == '2022Q1')].groupby(['quarter','Sector'])['Size_m'].sum().sort_values(ascending = False).to_frame().reset_index().rename(columns = {0: 'Amount'}).sort_values('Size_m')
sectors_q=sectors_q.sort_values('Size_m', ascending = False)

sectors_q = ig_deals_cleaned[(ig_deals_cleaned['quarter'] == '2021Q1') | (ig_deals_cleaned['quarter'] == '2022Q1')].groupby(['quarter','Sector'])['Size_m'].sum().sort_values(ascending = False).to_frame().reset_index().rename(columns = {0: 'Amount'}).sort_values('Size_m')
sectors_q=sectors_q.sort_values('Size_m', ascending = False)
sectors_graph = sectors_q.pivot(index='Sector', columns='quarter', values='Size_m').sort_values(by='2022Q1', ascending =False)

esg = ig_deals_cleaned.loc[(ig_deals_cleaned['ESG'] ==True) | (ig_deals_cleaned['ESG Type'].notnull()), ['PricingDate', 'quarter','month', 'year', 'week','DealIssuer', 'Sector', 'ESG Type', 'Size_m']].set_index('quarter')
esg_q_stats = esg.groupby(['quarter','ESG Type']).agg({'Size_m': 'sum'}).reset_index().set_index('quarter')

In [5]:
CHART_THEME = "plotly_white"
### TOP 10 SECTORS
# Create figure with secondary y-axis
sectors_fig = make_subplots(specs=[[{"secondary_y": True}]])


# Add traces
sectors_fig.add_trace(
    go.Bar(x=sectors_graph.head(10)['2022Q1']*1e5, y=sectors_graph.head(10).index, name="2022 Q1", orientation="h")
)
sectors_fig.add_trace(
    go.Bar(x=sectors_graph.head(10)['2021Q1']*1e5, y=sectors_graph.head(10).index,name="2021 Q1", orientation="h")
)
# Add figure title
sectors_fig.layout.template = CHART_THEME

sectors_fig.update_layout(
    barmode ="group",
    margin = dict(t=50, b=50, l=25, r=25)
)


# Set x-axis title
sectors_fig.update_xaxes(title_text="Sectors")
sectors_fig.update_yaxes(autorange="reversed")


sectors_fig.show()

In [6]:
###QUARTERLY VOLUMES

CHART_THEME = 'plotly_white'  # and try plotly_dark

chart_q = go.Figure()  # generating a figure that will be updated in the following lines
chart_q.add_trace(go.Bar(x=quarterly_stats.index, y=quarterly_stats['Size_m']*1e3,
                    name='Global Value'))
chart_q.layout.template = CHART_THEME
chart_q.layout.height=500
chart_q.update_layout(margin = dict(t=50, b=50, l=25, r=25))  # optm the chart space
chart_q.update_layout(
    #title='Quarterly Volume (USD $)',
    xaxis_tickfont_size=12,
    yaxis=dict(
        title='Value: $ USD',
        titlefont_size=14,
        tickfont_size=12,
        ))
# chart_ptfvalue.update_xaxes(rangeslider_visible=False)
# chart_ptfvalue.update_layout(showlegend=False)
chart_q.show()

In [7]:
### ESG VOLUMES
fig2 = go.Figure(data=[
   go.Bar(name='ESG_q', x=esg_q_stats.index, y=esg_q_stats['Size_m']*1e5)
    #go.Bar(name='SP500', x=plotlydf_portfval['date'], y=plotlydf_portfval['sp500_pctch'])
])


# Change the bar mode
fig2.update_layout(barmode='group')
fig2.layout.template = CHART_THEME
fig2.layout.height=300
fig2.update_layout(margin = dict(t=50, b=50, l=25, r=25))
fig2.update_layout(
#     title='% variation - Portfolio vs SP500',
    xaxis_tickfont_size=12,
    yaxis=dict(
        title='% change',
        titlefont_size=14,
        tickfont_size=12,
        ))
fig2.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.99))

fig2.show()

In [8]:
#COMPONENT4 TABLE
top10_ig = ig_deals_cleaned.loc[(ig_deals_cleaned['PricingDate'] >= '2022-01-01') & (ig_deals_cleaned['PricingDate'] <= '2022-03-31') & (ig_deals_cleaned['IssuerBorrowerType']!= 'FIG')].groupby(['PricingDate','DealIssuer', 'Sector' ])['Size_m', 'Book_Size'].sum().reset_index().sort_values('Size_m', ascending = False).nlargest(10, 'Size_m')
top10_ig.style.format({"PricingDate": "{:%Y-%m-%d}","Size_m": "{:,.0f}", "Book_Size": "{:,.0f}"})

top10_table = dash_table.DataTable(top10_ig.to_dict('rows'), [{"name": i, "id": i} for i in top10_ig.columns])


<ipython-input-8-5323ba1490d6>:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

C:\Users\U6033919\Anaconda3\lib\site-packages\pandas\core\frame.py:1549: FutureWarning:

Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.



In [9]:
fig_table = go.Figure(data=[go.Table(
    columnorder = [1,2,3,4,5],
    columnwidth = [80,120,120,90,90],
    header=dict(values=['Date' , 'Issuer', 'Sector', 'Amount US$', 'Books'],
                line_color='darkslategray',
                #fill_color='lightskyblue',
                align='left'),
    cells=dict(values=[['March 09' , 'March 21', 'March 07','March 02', 'March 04'], # 1st column
                       ['Magallanes' , 'GSK Consumer Healthcare', 'Rogers Communications','HCA', 'S&P Global'], # 2nd column
                       ['Spinco','Pharma','Telecom','Healthcare','Other'],
                       ['30,000', '8,750', '7,050', '6,000', '5,500'],
                       ['102,100', '33,750', '14,800', '-', '21,525']],
               
               line_color='darkslategray',
              # fill_color='lightcyan',
               align='left'))
])

fig_table.update_layout(width=800, height=500)
fig_table.layout.template = CHART_THEME
fig_table.show()

In [10]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.FLATLY])

app.layout = dbc.Container(
    [
        dbc.Row(dbc.Col(html.H2('DEBT TRENDS', className='text-center text-primary, mb-3'))),  # header row
        
        dbc.Row([  # start of second row
            dbc.Col([  # first column on second row
            html.H5('QUARTELY VOLUMES ($USD)', className='text-center'),
            dcc.Graph(id='chrt-portfolio-main',
                      figure=chart_q,
                      style={'height':550}),
            html.Hr(),
            ], width={'size': 7, 'offset': 0, 'order': 1}),  # width first column on second row
            dbc.Col([  # second column on second row
            html.H5('TOP TEN SECTORS', className='text-center'),
            dcc.Graph(id='indicators-ptf',
                      figure=sectors_fig,
                      style={'height':550}),
            html.Hr()
            ], width={'size': 5, 'offset': 0, 'order': 2}),  # width second column on second row
           # dbc.Col([  # third column on second row
           # html.H5('S&P500', className='text-center'),
          #  dcc.Graph(id='indicators-sp',
          #            figure=chart_q,
          #            style={'height':550}),
          #  html.Hr()
        #    ], width={'size': 2, 'offset': 0, 'order': 3}),  # width third column on second row
        ]),  # end of second row
        
        dbc.Row([  # start of third row
            dbc.Col([  # first column on third row
                html.H5('ESG SUSTAINABLE GREEN', className='text-center'),
                dcc.Graph(id='chrt-portfolio-secondary',
                      figure=fig2,
                      style={'height':380}),
            ], width={'size': 7, 'offset': 0, 'order': 1}),  # width first column on second row
            dbc.Col([  # second column on third row
                html.H5('TOP 5 ISSUES', className='text-center'),
                dcc.Graph(id='pie-top15',
                      figure = fig_table,
                      style={'height':380}),
            ], width={'size': 5, 'offset': 0, 'order': 2}),  # width second column on second row
        ])  # end of third row
        
    ], fluid=True)

if __name__ == '__main__':
    app.run_server(debug=False)

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Jul/2022 11:06:21] "GET /_alive_e53ad8ed-3745-4fa2-be6e-4f25c56f9cfd HTTP/1.1" 200 -


Dash app running on http://127.0.0.1:8050/


In [12]:
SIDEBAR_STYLE = {
    'position': 'fixed',
    'top': 0,
    'left': 0,
    'bottom': 0,
    'width': '12rem',
    'padding': '2rem 1rem',
    'background-color': 'lightgray',
}
CONTENT_STYLE = {
    'margin-left': '15rem',
    'margin-right': '2rem',
    'padding': '2rem' '1rem',
}

child = dbc.Container(
    [
        dbc.Row(dbc.Col(html.H2('DEBT TRENDS', className='text-center text-primary, mb-3'))),
        dbc.Row([
            dbc.Col([
            html.H5('QUARTER VOLUMES ($USD)', className='text-center'),
            dcc.Graph(id='chrt-portfolio-main',
                      figure=chart_q,
                      style={'height':550}),
            html.Hr(),

            ],
                width={'size': 7, 'offset': 0, 'order': 1}),
            dbc.Col([
            html.H5('TOP TEN SECTORS', className='text-center'),
            dcc.Graph(id='indicators-ptf',
                      figure=sectors_fig,
                      style={'height':550}),
            html.Hr()
            ],
                width={'size': 5, 'offset': 0, 'order': 2}),
           # dbc.Col([
          #  html.H5('S&P500', className='text-center'),
          #  dcc.Graph(id='indicators-sp',
          #            figure=indicators_sp500,
          #            style={'height':550}),
          #  html.Hr()
        #    ],
        #        width={'size': 2, 'offset': 0, 'order': 3}),
        ]),  # end of second row
        dbc.Row([
            dbc.Col([
                html.H5('ESG SUSTAINABLE GREEN', className='text-center'),
                dcc.Graph(id='chrt-portfolio-secondary',
                      figure=fig2,
                      style={'height':380}),
            ],
                width={'size': 7, 'offset': 0, 'order': 1}),
            dbc.Col([
                html.H5('TOP 5 ISSUES', className='text-center'),
                dcc.Graph(id='pie-top15',
                      figure = fig_table,
                      style={'height':380}),
            ],
                width={'size': 5, 'offset': 0, 'order': 2}),
        ])
        
    ], fluid=True)

sidebar = html.Div(
    [
#         html.H5("Navigation Menu", className='display-6'),
        html.Hr(),
        html.P('Navigation Menu', className='text-center'),
        
        dbc.Nav(
            [
                dbc.NavLink('Home', href="/", active='exact'),
                dbc.NavLink('Page2', href="/page-2", active='exact')
            ],
            vertical=True,
            pills=True,
        ),
    ],
    style=SIDEBAR_STYLE,
)

content = html.Div(id='page-content', children=child, style=CONTENT_STYLE)

app = JupyterDash(__name__, external_stylesheets=[dbc.themes.SPACELAB])
# app = JupyterDash(__name__)
app.layout = html.Div([
    dcc.Location(id='url'),
    sidebar,
    content
])

if __name__ == "__main__":
    app.run_server(debug=True, port=8056)

Dash app running on http://127.0.0.1:8056/


In [83]:
top10_ig

,PricingDate,DealIssuer,Sector,Size_m,Book_Size
108,2022-03-09,Magallanes Inc,Banking & Investment Services,30000.0,102100.0
123,2022-03-21,GSK Consumer Healthcare Capital US LLC,Pharmaceuticals & Medical Research,8750.0,33750.0
103,2022-03-07,Rogers Communications Inc,Telecommunications Services,7050.0,14800.0
93,2022-03-02,HCA Inc,Healthcare Services & Equipment,6000.0,0.0
59,2022-02-15,Bristol-Myers Squibb Co,Pharmaceuticals & Medical Research,6000.0,22600.0
101,2022-03-04,S&P Global Inc,Industrial & Commercial Services,5500.0,21525.0
100,2022-03-03,Roche Holding AG,Pharmaceuticals & Medical Research,5000.0,7500.0
130,2022-03-22,Lowe's Cos Inc,Retailers,5000.0,27500.0
102,2022-03-07,Berkshire Hathaway,Financial Technology (Fintech) & Infrastructure,4500.0,9700.0
70,2022-02-17,Amgen Inc.,Pharmaceuticals & Medical Research,4000.0,20000.0


In [84]:
fig_top10=ff.create_table(top10_ig)
fig_top10.show()

In [18]:
fig_top10.layout['annotations']

(layout.Annotation({
     'align': 'left',
     'font': {'color': '#ffffff'},
     'showarrow': False,
     'text': '<b>PricingDate</b>',
     'x': -0.45,
     'xanchor': 'left',
     'xref': 'x',
     'y': 0,
     'yref': 'y'
 }),
 layout.Annotation({
     'align': 'left',
     'font': {'color': '#ffffff'},
     'showarrow': False,
     'text': '<b>DealIssuer</b>',
     'x': 0.55,
     'xanchor': 'left',
     'xref': 'x',
     'y': 0,
     'yref': 'y'
 }),
 layout.Annotation({
     'align': 'left',
     'font': {'color': '#ffffff'},
     'showarrow': False,
     'text': '<b>Sector</b>',
     'x': 1.55,
     'xanchor': 'left',
     'xref': 'x',
     'y': 0,
     'yref': 'y'
 }),
 layout.Annotation({
     'align': 'left',
     'font': {'color': '#ffffff'},
     'showarrow': False,
     'text': '<b>Size_m</b>',
     'x': 2.55,
     'xanchor': 'left',
     'xref': 'x',
     'y': 0,
     'yref': 'y'
 }),
 layout.Annotation({
     'align': 'left',
     'font': {'color': '#ffffff'},
     'sh

In [85]:
new_layouts = {
    -0.45: -0.45,
    0.55: 0.05,
    1.55: 1.00,
    2.55: 2.45,
    3.55: 5.50
}

In [79]:
fig_top10.show()

In [86]:
fig_top10.layout['xaxis']

layout.XAxis({
    'dtick': 1, 'gridwidth': 2, 'showticklabels': False, 'tick0': -0.5, 'ticks': '', 'zeroline': False
})

In [87]:
fig_top10.layout['xaxis']['tickmode'] = 'array'
fig_top10.layout['xaxis']['tickvals'] = [x-0.05 for x in new_layouts.values()]

In [88]:
fig_top10.show()